In [2]:
from phe import paillier
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.feature_selection import mutual_info_classif

# ------------------------------
# Data Preprocessing for UNSW-NB15
# ------------------------------


def load_and_preprocess_data(path):
    df = pd.read_csv(path)

    # Drop unnecessary columns
    #columns_to_drop = ['id', 'attack_cat']
    #for col in columns_to_drop:
    #    if col in df.columns:
    #        df.drop(columns=[col], inplace=True)

    # Handle missing values
    df.replace(['-'], np.nan, inplace=True)
    df.fillna(method='ffill', inplace=True)

    # Encode object-type columns
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype.name == 'category':
            encoder = LabelEncoder()
            df[col] = encoder.fit_transform(df[col].astype(str))

    # Undersample the majority class (label = 1)
    class_0 = df[df['label'] == 0]
    class_1 = df[df['label'] == 1]

    # Downsample class 1 to match class 0
    class_1_sampled = class_1.sample(n=len(class_0), random_state=42)

    # Combine balanced dataset
    df_balanced = pd.concat([class_0, class_1_sampled]).sample(frac=1, random_state=42)

    y = df_balanced['label'].values
    X = df_balanced.drop(columns=['label'])

    # Feature selection using Mutual Information
    mi_scores = mutual_info_classif(X, y, discrete_features='auto')
    mi_series = pd.Series(mi_scores, index=X.columns)
    selected_features = mi_series[mi_series >= 0.1].index.tolist()

    print(f"✅ Selected features based on MI >= 0.1: {selected_features}")
    X_selected = X[selected_features].values

    # Normalize features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_selected)

    return X_scaled, y


# ------------------------------
# Client Class
# ------------------------------

class Client:
    def __init__(self, id, public_key, private_key, data, labels):
        self.id = id
        self.public_key = public_key
        self.private_key = private_key

        # Split into local train/test
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            data, labels, test_size=0.3, random_state=42, stratify=labels
        )

        self.model_weights = None

    def train_local_model(self):
        clf = SGDClassifier(loss='log_loss', max_iter=1000, learning_rate='constant', eta0=0.01)
        clf.fit(self.X_train, self.y_train)
        self.model_weights = clf.coef_[0]
        return self.model_weights

    def encrypt_weights(self):
        return [self.public_key.encrypt(x) for x in self.model_weights]

    def decrypt_weights(self, encrypted_weights):
        return np.array([self.private_key.decrypt(x) for x in encrypted_weights])


# ------------------------------
# Server Class
# ------------------------------

class Server:
    def __init__(self, public_key):
        self.public_key = public_key
        self.encrypted_models = []

    def receive_encrypted_model(self, encrypted_model):
        self.encrypted_models.append(encrypted_model)

    def aggregate_encrypted_models(self, num_clients):
        n_weights = len(self.encrypted_models[0])
        aggregated_model = []
        for i in range(n_weights):
            sum_enc = sum(client_weights[i] for client_weights in self.encrypted_models)
            avg_enc = sum_enc * (1 / num_clients)
            aggregated_model.append(avg_enc)
        return aggregated_model


# ------------------------------
# Federated Training Function
# ------------------------------

def federated_training(data_path, num_clients=3, num_rounds=3):
    X, y = load_and_preprocess_data(data_path)

    # Stratified split across clients to maintain label balance
    skf = StratifiedKFold(n_splits=num_clients, shuffle=True, random_state=42)
    split_data, split_labels = [], []
    for _, idx in skf.split(X, y):
        split_data.append(X[idx])
        split_labels.append(y[idx])

    public_key, private_key = paillier.generate_paillier_keypair()

    clients = [Client(i, public_key, private_key, split_data[i], split_labels[i]) for i in range(num_clients)]
    server = Server(public_key)

    for round in range(num_rounds):
        print(f"\n🔁 Round {round + 1} - Local training and encryption")

        server.encrypted_models.clear()

        for client in clients:
            local_weights = client.train_local_model()
            enc_weights = client.encrypt_weights()
            server.receive_encrypted_model(enc_weights)
            print(f"✅ Client {client.id} encrypted and sent weights.")

        print("🔐 Server aggregating encrypted models...")
        encrypted_global_model = server.aggregate_encrypted_models(num_clients)

        for client in clients:
            decrypted_global_model = client.decrypt_weights(encrypted_global_model)
            client.model_weights = decrypted_global_model
            print(f"📥 Client {client.id} decrypted global model.")

    return clients


# ------------------------------
# Evaluation
# ------------------------------

def evaluate_global_model(clients):
    for client in clients:
        clf = SGDClassifier()
        clf.coef_ = client.model_weights.reshape(1, -1)
        clf.intercept_ = np.array([0])
        clf.classes_ = np.array([0, 1])  # Assuming binary classification

        preds = clf.predict(client.X_test)
        acc = accuracy_score(client.y_test, preds)
        print(f"🎯 Client {client.id} Test Accuracy: {acc:.2f}")


# ------------------------------
# Run the Whole Pipeline
# ------------------------------

clients = federated_training("C:\\MAFL\\MAFL\\unsw-nb15-training-set.csv")
evaluate_global_model(clients)


C:\Users\sowmi\AppData\Local\Temp\ipykernel_16880\197836737.py:26: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)


✅ Selected features based on MI >= 0.1: ['dur', 'proto', 'state', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit', 'djit', 'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'attack_cat']

🔁 Round 1 - Local training and encryption
✅ Client 0 encrypted and sent weights.
✅ Client 1 encrypted and sent weights.
✅ Client 2 encrypted and sent weights.
🔐 Server aggregating encrypted models...
📥 Client 0 decrypted global model.
📥 Client 1 decrypted global model.
📥 Client 2 decrypted global model.

🔁 Round 2 - Local training and encryption
✅ Client 0 encrypted and sent weights.
✅ Client 1 encrypted and sent weights.
✅ Client 2 encrypted and sent weights.
🔐 Server aggregating encrypted models...
📥 Client 0 decrypted global model.
📥 Client 1 decrypted global model.
📥 Client 2 decrypted global model.

🔁 Round 3 - Local training and encryption
✅ Client 0 encrypted and sent weights.
✅ Client 1 encrypted